In [13]:
#%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Definición de Librería

In [5]:
import tweepy
import json
import pandas as pd
import time
import GetOldTweets3 as got


In [42]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb


In [7]:
pd.set_option('display.max_rows', 1000)

In [8]:
#Definción de variables
df_followers_id = pd.DataFrame()
df_followers_info = pd.DataFrame()


# Definición de las claves para acceder a Twitter

In [9]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [10]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

# Extracción de los datos de la cuenta

In [38]:
def extract_info_account():
    item = api.get_user('@elcorteingles')
    data_account=[]
    account={}
    account["name"]=item.name
    account["id"]=item.id
    account["screen name"]=item.screen_name
    account["friends_count"]=item.friends_count
    account["followers_count"]=item.followers_count
    data_account.append(account)
    df_account_info = pd.DataFrame(data_account)
    return df_account_info
    
df_account_info = extract_info_account()
df_account_info

,name,id,screen name,friends_count,followers_count
0,El Corte Inglés,262541531,elcorteingles,1628,291318


# Followers

# Extracción de las cuentas Followers

In [12]:
#Nueva forma de obtener todas las cuentas que siguen a El Corte Inglés en España

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_followers_ids(api, twitter_account='@elcorteingles'):
    while True:
        try:
            for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
                followers_ids.extend(page)
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return followers_ids

followers_ids=[]
followers_ids=get_followers_ids(api)
#Comprobación de que el número de followers es el correcto
#print(len(followers_ids))
#Guardado de los ids en los ficheros para agilizar el desarrollo
df_followers_id= pd.DataFrame(followers_ids)
df_followers_id.columns = ['id_Follower']
df_followers_id

Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893


KeyboardInterrupt: 

# Definición de tramos de las cuentas Followers

In [ ]:
#Agrupación de ID por la limitación de la API de Twitter
followers_limit=[df_followers_id[n:n+99] for n in range(0, len(df_followers_id), 100)]
print(len(followers_limit))

# Extracción de la información de las cuentas Followers

In [ ]:
# Existe limitación al traer la información de los seguidores
data = []
for followers in followers_limit:
    follower_obj=api.lookup_users(user_ids=followers) 
    for user in follower_obj:
        follower={}
        follower["id"]=user.id
        follower["name"]=user.name
        follower["screen_name"]=user.screen_name
        follower["created_at"]=user.created_at
        follower["location"]=user.location
        follower["followers_count"]=user.followers_count
        follower["friends_count"]=user.friends_count
        follower["description"]=user.description
        data.append(follower)

df_followers_info = pd.DataFrame(data)
df_followers_info

In [ ]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file, index=False)

saveID(df_followers_id,'CSVs/followers_id_el_corte_ingles.csv')
saveID(df_followers_info,'CSVs/df_followers_info_el_corte_ingles.csv')

# Followings

# Extracción de las cuentas Followings

In [34]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@elcorteingles'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids

following_ids=[]
following_ids=get_following_ids(api)
following_ids
df_following_id= pd.DataFrame(following_ids)
df_following_id.columns = ['id_Following']
df_following_id

,id_Following
0,252096294
1,940274906025091073
2,2236669338
3,570041347
4,1041739380455624704
5,182764833
6,200081186
7,1042007718553415682
8,197399746
9,5494572


# Definición de tramos de las cuentas Following

In [28]:
#Agrupación de ID por la limitación de la API de Twitter
following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
print(len(following_limit))

17


In [29]:
print(following_limit)

[[252096294, 940274906025091073, 2236669338, 570041347, 1041739380455624704, 182764833, 200081186, 1042007718553415682, 197399746, 5494572, 1728956976, 1095345891542736897, 403396902, 62591415, 14712261, 1042012837319770114, 981893453142548480, 288787209, 505763621, 1097413622517690370, 934833810, 167481898, 170819033, 16684359, 2827858225, 383294819, 847017228033806336, 4044880565, 1734700669, 334684429, 2251552290, 2978762543, 177559871, 2269563059, 4888068429, 2174698591, 97654669, 2174859445, 17209618, 92281182, 1060678345808068608, 955613952, 804617603012501504, 259824220, 1946527747, 827527807, 2172740569, 283699138, 2989465757, 1047037402215268352, 306112241, 42050597, 198787442, 1379710231, 3392188757, 978000108183834624, 1951504608, 117748602, 574649962, 889307215, 1132246848, 202092114, 1501434991, 364968471, 2480087774, 921689998507827202, 986550461850443778, 856451172416311296, 2183187386, 1024860452, 18736723, 1018495825, 1325475392, 710144100679421953, 3378657623, 3045383

# Extracción información de las cuentas Followings

In [35]:
# Existe limitación al traer la información de los seguidores
def get_info_following():
    data = []
    for followings in following_limit:
        following_obj=api.lookup_users(user_ids=followings) 
        for user in following_obj:
            following={}
            following["id"]=user.id
            following["name"]=user.name
            following["screen_name"]=user.screen_name
            following["created_at"]=user.created_at
            following["location"]=user.location
            following["followers_count"]=user.followers_count
            following["friends_count"]=user.friends_count
            following["description"]=user.description
            data.append(following)
    return data

following_info =[]
following_info= get_info_following()
df_following_info = pd.DataFrame(following_info)
df_following_info

[252096294, 940274906025091073, 2236669338, 570041347, 1041739380455624704, 182764833, 200081186, 1042007718553415682, 197399746, 5494572, 1728956976, 1095345891542736897, 403396902, 62591415, 14712261, 1042012837319770114, 981893453142548480, 288787209, 505763621, 1097413622517690370, 934833810, 167481898, 170819033, 16684359, 2827858225, 383294819, 847017228033806336, 4044880565, 1734700669, 334684429, 2251552290, 2978762543, 177559871, 2269563059, 4888068429, 2174698591, 97654669, 2174859445, 17209618, 92281182, 1060678345808068608, 955613952, 804617603012501504, 259824220, 1946527747, 827527807, 2172740569, 283699138, 2989465757, 1047037402215268352, 306112241, 42050597, 198787442, 1379710231, 3392188757, 978000108183834624, 1951504608, 117748602, 574649962, 889307215, 1132246848, 202092114, 1501434991, 364968471, 2480087774, 921689998507827202, 986550461850443778, 856451172416311296, 2183187386, 1024860452, 18736723, 1018495825, 1325475392, 710144100679421953, 3378657623, 30453839

[35743259, 230567961, 108908254, 294769333, 1588486417, 316756361, 807608628, 125416477, 106876550, 332378234, 105128028, 597554268, 1071338250, 339049117, 525823798, 277545723, 111689357, 160968073, 29823160, 1614411, 496278040, 173758793, 16883794, 556719361, 322571370, 182769742, 146597415, 105451020, 256525164, 19394390, 210067976, 134609720, 411510170, 144883098, 249187529, 40255499, 308388712, 565774540, 372296516, 38213723, 320647362, 174391424, 273643366, 46407069, 469478156, 210786658, 561276903, 538915993, 354256296, 318531174, 88749392, 533287451, 71503742, 187458738, 127648692, 268456652, 353005590, 186982547, 712068680, 54204575, 2150777605, 214857544, 36023053, 84182932, 230537167, 72988880, 176511185, 2397702889, 1125017588, 125626568, 225834298, 381972447, 254103212, 1094558022, 263131563, 105082141, 2729374621, 927471277, 2729358361, 2729352572, 2729372558, 2729396395, 2729161494, 242685330, 907925701, 195712499, 491490393, 18480059, 34507480, 1465608990, 85563781, 129

[69008563, 42856346, 16808626, 164759704, 436628301, 148516500, 39727912, 343973305, 212265984, 607281049, 182889996, 66789097, 33173981, 313245901, 67995848, 27200555, 76010207, 12732492, 823059337, 399521071, 202561591, 414893224, 488410824, 16293385, 164228101, 414886496, 207536155, 399520547]


,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,252096294,Dani García,danigarcia_ca,2011-02-14 13:48:18,Marbella,126889,2971,"Cocinero desde los 17, 3*** Michelin y @Bibodanigarcia Marbella/Madrid/Doha/Tarifa @lobitodemardg Marbella/Madrid @hacerdecomertve"
1,940274906025091073,Javier Perez Pallares #Neumologia,jpallaresneumo,2017-12-11 17:39:41,"Cartagena, España",1930,866,#Neumologia. Interventional pulmonologist.Vicepresidente Asociacion Española Endoscopia Respiratoria y Neumologia Intervencionista. CEO @aerisrespira.
2,2236669338,Pedro Mozas Rello,pedromozasrello,2013-12-08 22:39:01,Madrid,303,376,"Publicista, escritor, guionista, blogger. Miembro del Club de Creativos de España. Autor de ""Incorrupto"" (Ed. Distrito 93 - @BooksBunker)."
3,570041347,Ana de Armas,Ana_d_Armas,2012-05-03 14:59:50,,238382,445,Artist.
4,1041739380455624704,alba reche,_albxreche,2018-09-17 17:23:17,España,208755,299,quimera https://t.co/YL3CeV0U9L
5,182764833,Metro de Madrid,metro_madrid,2010-08-25 10:18:32,"Madrid, España",559674,541,Metro de Madrid es la empresa pública que gestiona la red de metro de la Comunidad de Madrid.\r\nhttp://t.co/oKICiy53
6,200081186,Rocío Gil Grande,pepi_rocio,2010-10-08 12:02:33,Madrid,919,651,Periodista y feminista. Este perfil es personal.
7,1042007718553415682,Carlos Right,carlos_right,2018-09-18 11:09:34,"Barcelona, España",100165,244,#prisionero💘
8,197399746,Andy Lucas oficial,_AndyyLucas_,2010-10-01 11:58:43,España,115992,30050,"más de dos millones de discos vendidos,más de 100 premios nacionales e internacionales,más de mil conciertos en toda su carrera y cuatro millones de asistentes."
9,5494572,mtvspain,mtvspain,2007-04-25 11:30:20,"Madrid, Spain",2806382,914,"TV 📺 música 🎧 actualidad 🤗y más 🙌 Puedes vernos en Movistar+(24),Vodafone TV(153),Orange TV(60), Sky, Fubo TV, Virgin, Telecable(18),R(31), K(23) y #MTVPLAY"


In [48]:
file = File_Manage()
fl_account_info = 'CSVs/info_el_corte_ingles.csv'
file.setFile(fl_account_info)
file.writeCSV(df_account_info,file.getFile())
fl_following_id = 'CSVs/following_id_el_corte_ingles.csv'
file.setFile(fl_following_id)
file.writeCSV(df_following_id,file.getFile())
fl_following_info = 'CSVs/following_info_el_corte_ingles.csv'
file.setFile(fl_following_info)
file.writeCSV(df_following_info,file.getFile())
